In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [3]:
df = pd.read_csv('streamer_df')

In [4]:
df

,Unnamed: 0,Unnamed: 1,username,AVG Viewers,Time Streamed,All Time Peak Viewers,Hours Watched,Rank,Followers Gained,Total Followers,Total Views
0,#1,NaN,tommyinnit,195782,14.3hours,650237,2806209,2,334307,5499713,49185807
1,#2,NaN,Quackity,158364,10.2hours,381550,1620588,4,198959,3471874,23178122
2,#3,NaN,Sapnap,129001,9.2hours,287152,1188960,8,183191,2237876,8740493
3,#4,NaN,auronplay,113520,109.4hours,313347,12419097,3,341562,8741858,142994817
4,#5,NaN,xQcOW,109059,339.9hours,222720,37072768,1,182238,5776761,361856811
...,...,...,...,...,...,...,...,...,...,...,...
2445,#2446,NaN,SwordsofLegendsOnline,1237,18.7hours,4074,23158,3299,5165,11966,89491
2446,#2447,NaN,easywithaces,1235,100.4hours,18686,124074,2498,1685,111557,8879377
2447,#2448,NaN,SimpleFlips,1234,64.1hours,8607,79125,2842,1877,139942,5155937
2448,#2449,NaN,Raptor,1233,148hours,5270,182520,2196,4138,171559,3312085


In [10]:
# creates a dataframe for social media plattforms scraped from twitch for a given df with usernames in it
def get_social(df):
    ''' this function scrapes the URL to youtube and twitter for a username-list'''
    import time
    t=5
    usernames = df.username.to_list()
    options = webdriver.FirefoxOptions()
    options.binary_location = "/Applications/Firefox.app"
    chrome_driver_binary = "/Users/ludwigmehmkrre/drivers/firefoxdriver"
    dict = {}
    for user in usernames:
        URL= f'https://www.twitch.tv/{user}/about'
        driver = webdriver.Firefox(chrome_driver_binary, chrome_options=options)
        driver.get(URL)
        soup=BeautifulSoup(driver.page_source)
        twitter = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "twitter.com/" in a["href"]]))
        youtube = list(set([a["href"] for a in soup.find_all("a", href=True) if "youtube.com/" in a["href"]]))
        instagram = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "instagram" in a["href"]]))
        discord = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "discord" in a["href"]]))
        if twitter:
            twitter = twitter[0]
        if instagram:
            instagram = instagram[0]
        if discord:
            discord = discord[0]
        dict[user] = [twitter, youtube, instagram, discord]
        time.sleep(t)
    df = pd.DataFrame.from_dict(dict, orient='index')
    df['Twitter'] = df[0].apply(lambda x: None if x == [] else x)
    df['YouTube'] = df[1].apply(lambda x: None if x == [] else x)
    df['Instagram'] = df[2].apply(lambda x: None if x == [] else x)
    df['Discord'] = df[3].apply(lambda x: None if x == [] else x)
    df.index.name = 'username'
    df = df.reset_index()
    return df[['username', 'Twitter', 'YouTube', 'Instagram', 'Discord']]

In [11]:
socials = get_social(df)
df_socs = socials.copy()

TypeError: __init__() got an unexpected keyword argument 'chrome_options'

In [9]:
def get_social_id(link):
    '''Extract streamer identifier at the end of their social media links'''
    if link[-1] == '/':
        link = link[:-1]
    start = link.rfind('/')
    id = link[start + 1:]

    return id

def get_yt_channel_id(links):
    '''Return the first genuine yt channel id in a list of yt links'''
    if not links:
        return None
    channel_id = None
    for link in links:
        link = link.split('?')[0]
        print(link)
        id = get_social_id(link)
        if len(id) == 24:
            channel_id = id
            break
    
    return channel_id


def parse_streamer_socials(socials_df):
    '''Return a dataframe with relevant ids per treamer and platform'''
    for (column_name, column_data) in socials_df.iteritems():
        if column_name != 'Discord':
            if column_name == 'YouTube':
                socials_df[column_name] = socials_df[column_name].apply(get_yt_channel_id)
            socials_df[column_name] = column_data.apply(lambda x: get_social_id(x) if x else None)
            
        
    return socials_df

In [ ]:
get_yt_channel_id(test_link)

In [ ]:
parse_streamer_socials(df_socs)